In [1]:
%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import netCDF4
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from scipy.integrate import simps


import createFolder as cF


In [2]:
year = '2016'
month = '12'
day = '20'
stn_lat = 59.8
stn_lon = 7.2

In [3]:
def mask_array(variable,ens_memb,y,x):
    if np.ma.is_masked(variable[:,:,ens_memb,y[0],x[0]]):
        mask = np.ma.getmaskarray(variable[:,:,ens_memb,y[0],x[0]])  
        fill_value = np.nan
        marr = np.ma.array(variable[:,:,ens_memb,y[0],x[0]], mask = mask, fill_value = fill_value)
        dtype = marr.filled().dtype
    else:
        fill_value = np.nan
        marr = variable[:,:,ens_memb,y[0],x[0]]
        dtype = marr.dtype
    return(marr.filled())

In [4]:
def get_value_at_station(val):
    array = pd.DataFrame.from_dict(val[:,:])
#    val_ml = pd.DataFrame.from_dict(val_ml[:,:,ens_memb,y[0],x[0]])
    return(array)

In [ ]:
def get_microphy_val(var_ml,ens_memb):
    var_ml = fn.variables[var_ml]
    var_ml = mask_array(var_ml,ens_memb,y,x)
    var_ml = get_value_at_station(var_ml)
    var_ml = np.fliplr(var_ml)
    return(var_ml);

In [5]:
thredds  = 'http://thredds.met.no/thredds/dodsC/meps25epsarchive'
time = '00'
fn = netCDF4.Dataset('%s/%s/%s/%s/meps_full_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,time),'r')


## Latitudes
## [y = 949][x = 739]
latitude = fn.variables['latitude']

## Longitudes 
## [y = 949][x = 739]
longitude = fn.variables['longitude']



In [6]:
# Now find the absolute value of the difference between the  station's lat/lon with every point in the grid. 
# This tells us how close a point is to the particular latitude and longitude.
abslat = np.abs(latitude[:,:]-stn_lat)
abslon= np.abs(longitude[:,:]-stn_lon)

# Now we need to combine these two results. We will use numpy.maximum, which takes two arrays and finds the local 
# maximum.
c = np.maximum(abslon, abslat)

# If you don't like flattened arrays, you can also get the row/column index like this
y, x = np.where(c == np.min(c))

In [7]:
######## with Vertical Levels ( height0 ) #################################
air_temperature_0m = fn.variables['air_temperature_0m']
surface_air_pressure = fn.variables['surface_air_pressure']


In [8]:
######## with Vertical Levels ( hybrid )#################################
## hybrid levels atmosphere_hybrid_sigma_pressure_coordinate
# formula: p(n,k,j,i) = ap(k) + b(k)*ps(n,j,i)
# positive: down
air_temperature_ml= fn.variables['air_temperature_ml']
### variables to calculate pressure
p0 = fn.variables['p0']    ## p0: p0
ap = fn.variables['ap']    ## ap: ap
b = fn.variables['b']      ## b: b

In [9]:
ens_memb = 2

In [10]:
### Connect model levels and surface values
# this is only done for pressure and temperature
# !!!! DO NOT DO THAT FOR THE OTHER VALUES, SINCE in model levels it is the mixing ration and at the surface [kg/m^2]

### Pressure
# transform hybrid sigma pressure coordinate in model levels to actual pressure
surface_air_pressure = mask_array(surface_air_pressure,ens_memb,y,x)
p_sfc = get_value_at_station(surface_air_pressure) 
p_ml = pd.DataFrame.from_dict((ap[:] + b[:]*surface_air_pressure[:,:]))
pressure = np.asarray(pd.concat([p_ml, p_sfc],axis = 1))


In [11]:
### Temperature
air_temperature_0m = mask_array(air_temperature_0m,ens_memb,y,x)
air_temperature_ml = mask_array(air_temperature_ml,ens_memb,y,x)
t_sfc = get_value_at_station(air_temperature_0m)
t_ml = get_value_at_station(air_temperature_ml)
temperature = np.asarray(pd.concat([t_ml,t_sfc],axis = 1))

### Snowfall_amount
#    snowfall_amount_0m = get_value_at_station(snowfall_amount_0m)
 #   snowfall_amount_ml = get_value_at_station(snowfall_amount_ml)

In [12]:
##### to convert pressure-levels into actual heights use the hypsometric equation --> Temperature and pressure
# are needed. After J. E. Martin: Mid-Latitude Atmospheric Dynamics Eq. 3.6
Rd = 287.    # gas constant for dry air [J kg^-1 K^-1]
g = 9.81     # Standard gravity [m s^-2]

### calculate the pressure-weighted, column averaged temperature as from J. E. Martin: Mid-Latitude Atmospheric 
# Dynamics Book, Eq. below Eq. 3.6
temp_mean = []
for i in range(0, temperature.shape[1]):
 ##       t = np.asarray(t_sfc[0])
   #     numT = simps(y=t[:,i:(i+2)], x=pressure[:,i:(i+2)], dx = np.log(pressure[:,i:(i+2)]))
    numT = simps(y=temperature[:,i:(i+2)], x=pressure[:,i:(i+2)], dx = np.log(pressure[:,i:(i+2)]))
    denomT = simps(y=np.ones(temperature[:,i:(i+2)].shape), x = pressure[:,i:(i+2)], dx = np.log(pressure[:,i:(i+2)]))
    t_mean = numT/denomT
    temp_mean.append(t_mean)

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


In [13]:
# get temperature and pressure, and value so that array zero contains low levels (transpose or flip)
temp_mean = pd.DataFrame.from_dict(np.transpose((temp_mean)))
temp_mean = np.fliplr(temp_mean)
pres = np.fliplr(pressure)
temperature = np.fliplr(temperature)
#snowfall_amount_0m = np.fliplr(snowfall_amount_0m)
#snowfall_amount_ml = np.fliplr(snowfall_amount_ml)

In [14]:
thickness = []
geop_th = []
for i in range(0, pres.shape[1]):
    if (i+1) == pres.shape[1]:
        continue
    p1 = pres[:,i]
    p2 = pres[:,(i+1)]   
    dz = (Rd * temp_mean[:,i])/g * np.log((p1/p2))    # thickness in [m]
    dgeop = (Rd * temp_mean[:,i])* np.log((p1/p2))    # thickness in [J/kg]
    thickness.append(dz)
    geop_th.append(dgeop)
    



In [15]:
# transpose array, so that array starts at lower levels
thickness = pd.DataFrame.from_dict(np.transpose((thickness)))
geop_th = pd.DataFrame.from_dict(np.transpose((geop_th)))

In [ ]:
# Read in all other values
## Time
time = fn.variables['time']
## heights
height0 = fn.variables['height0']
height1 = fn.variables['height1']
hybrid = fn.variables['hybrid']


In [ ]:
######## with Vertical Levels ( height0 ) #################################
air_temperature_0m = get_microphy_val('air_temperature_0m',ens_memb)
graupelfall_amount = get_microphy_val('graupelfall_amount',ens_memb)
liquid_water_content_of_surface_snow = get_microphy_val('liquid_water_content_of_surface_snow',ens_memb)
precipitation_amount_acc = get_microphy_val('precipitation_amount_acc',ens_memb)
rainfall_amount = get_microphy_val('rainfall_amount',ens_memb)
snowfall_amount = get_microphy_val('snowfall_amount',ens_memb)
surface_air_pressure = get_microphy_val('surface_air_pressure',ens_memb)


#surface = [
 #         air_temperature_0m,                      ## surface temperature
  #        graupelfall_amount,                      ## graupelfall amount
   #       liquid_water_content_of_surface_snow,    ## Snow water equivalent
    #      precipitation_amount_acc,                ## accumulated  total precipitation
     #     rainfall_amount,                         ## instantanous rainfall at surface
      #    snowfall_amount,                         ## instantaneous snowfall amount at surface
       #   surface_air_pressure                     ## ps: surface_air_pressure
        #       ]



In [ ]:
######## with Vertical Levels ( height1 )#################################
air_temperature_2m = get_microphy_val('air_temperature_2m',ens_memb)
specific_humidity_2m = get_microphy_val('specific_humidity_2m',ens_memb)
#two_meter = [
 #            air_temperature_2m,                   ## screen level temperature
  #           specific_humidity_2m                  ## screen level specific humidity
   #         ]

In [ ]:
######## with Vertical Levels ( hybrid )#################################
## hybrid levels atmosphere_hybrid_sigma_pressure_coordinate
# formula: p(n,k,j,i) = ap(k) + b(k)*ps(n,j,i)
# positive: down
air_temperature_ml= get_microphy_val('air_temperature_ml',ens_memb)
atmosphere_cloud_condensed_water_content_ml = get_microphy_val('atmosphere_cloud_condensed_water_content_ml',ens_memb)
atmosphere_cloud_ice_content_ml = get_microphy_val('atmosphere_cloud_ice_content_ml',ens_memb)
graupelfall_amount_ml = get_microphy_val('graupelfall_amount_ml',ens_memb)
pressure_departure = get_microphy_val('pressure_departure',ens_memb)
rainfall_amount_ml = get_microphy_val('rainfall_amount_ml',ens_memb)
snowfall_amount_ml = get_microphy_val('snowfall_amount_ml',ens_memb)
specific_humidity_ml = get_microphy_val('specific_humidity_ml',ens_memb)
            
            
#model_levels = [
 #              air_temperature_ml,                            ## air temperature model levels
  #             atmosphere_cloud_condensed_water_content_ml,   ## atmospheric cloud condensed water
   #            atmosphere_cloud_ice_content_ml,               ## cloud ice in model levels
    #           graupelfall_amount_ml,                         ## instantaneous graupel in model levels
     #          pressure_departure,                            ## nonhydrostatic departure from hydrostatic pressure
      #         rainfall_amount_ml,                            ## instantaneous rain in model levels
       #        snowfall_amount_ml,                            ## instantaneous snow in model levels
        #       specific_humidity_ml                           ## specific humidity model levels
         #      ]



In [ ]:
### write netCDF file
dirnc = '../../MEPS_data'
f = netCDF4.Dataset('%s/%s%s%s_%s.nc' %(dirnc, year, month, day, time), 'w')

### create dimensions
f.createDimension('time', )